In [1]:
#System and standard inputs
import sys
import importlib
import timeit
import time
import collections

#Adding correct paths to custom modules
sys.path.append('/Users/sasrey/AlphaTrade/gym_exchange')
sys.path.append('/Users/sasrey/AlphaTrade')
sys.path.append('/Users/sasrey/AlphaTrade/gymnax_exchange')

In [2]:
#Relevant Jax Imports
import jax.numpy as jnp
from jax import lax
import jax

In [355]:
#Direct Import of the Orderbook functions 
import jaxob.JaxOrderbook as job 
job=importlib.reload(job)
#OrderBook class import as written by Kang
import orderbook.orderbook
importlib.reload(orderbook.orderbook)
from orderbook.orderbook import OrderBook as cpuOrderBook
#Import (and reload for debugging) of the OrderBook class with jax backend (SF) should behave in the same way.
import jorderbook
importlib.reload(jorderbook)
from jorderbook import OrderBook as JaxOrderBook

#Import Data Mgt and Encoding/Decoding tools
#(The aim of these is to adjust for the case where no message is received but data ends up in the L2 data)
import gym_exchange.data_orderbook_adapter.data_pipeline as dpl
from gym_exchange.data_orderbook_adapter.decoder import Decoder
from gym_exchange.data_orderbook_adapter.encoder import Encoder

#Attempt at using the data pipeline tools and the encoder/decoder, doesn't really work well. 

dplOBJ=dpl.DataPipeline()
decodeddata=Decoder(10,40,dplOBJ.historical_data,dplOBJ.data_loader)
#Decoder Functionality: Takes the message data (dataloader) and the orderbook data (historical data).
#Creates and maintains an orderbook as implemented in the CPU version.
#Creates and maintains a "DataAdjuster" object, which considers the historical data (orderbook) to be the "true" state
#Inits the orderbook with the first state in the orderbook data (hist) gives imaginary list of IDs to the orders created:
#each entry in the orderbook is considered to be from a single order. 
output=decodeddata.process()
#Process function for Decoder: goes through N (horizon) messages to ascertain whether there are adidtional messages to consider. 

enc=Encoder(decodeddata)
#enc()

In [4]:
import pandas as pd
df_msg = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_message_50.csv", header=None)
df_ob = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_orderbook_50.csv", header=None)
orderbookLevels=50
df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})

In [5]:
data=jnp.array(df_ob.iloc[0,:]).reshape(int(orderbookLevels*2),2) ##Careful - converts to int32 in jnp array, meaning the filler values get truncated.
newarr=jnp.zeros((int(orderbookLevels*2),8))
initOB=newarr.at[:,3].set(data[:,0]).at[:,2].set(data[:,1]).at[:,0].set(1).at[0:orderbookLevels*4:2,1].set(-1).at[1:orderbookLevels*4:2,1].set(1).at[:,4].set(0).at[:,5].set(job.INITID).at[:,6].set(34200).at[:,7].set(0).astype('int32')


df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})
df_orders['TradeID']=0
col_order=['Type','Side','Quantity','Price','TradeID','OrderID','Time']
df_orders=df_orders[col_order]
df_orders_simple=df_orders[(df_orders['Type']!=6)&(df_orders['Type']!=7)&(df_orders['Type']!=5)]


In [6]:
test=df_orders["Time"].astype('string').str.split('.',expand=True)
df_orders[["TimeWhole","TimeDec"]]=test.astype('int32')#.rename(['TimeWhole','TimeDec'])
df_orders=df_orders.drop("Time",axis=1)
ordersJNP=jnp.array(df_orders)
print(ordersJNP.shape)

(290737, 8)


In [371]:
#horizon=14137
horizon=1
orderbook_1=JaxOrderBook(price_levels=150,orderQueueLen=100)
trades_init=orderbook_1.process_orders_array(initOB)
t1=time.time()
trades_msgs=orderbook_1.process_orders_array(ordersJNP[0:horizon]).block_until_ready()
t2=time.time()
#trades_msgs=orderbook_1.process_orders_array(ordersJNP[horizon-1:horizon])
print(t2-t1)

2.092087984085083


In [304]:
MINPRICE=-1.410065407e9
myOB=orderbook_1.get_L2_state()
L2OB=jnp.array(df_ob.iloc[horizon-1])
L2OB=L2OB.at[L2OB==MINPRICE].set(-1)

print(ordersJNP[horizon-1])
print('#'*30+'JAX - L2 DATA LOB Difference for Horizon: '+ str(horizon)+' '+'#'*30)
print((myOB.reshape(150,4)[0:50,:]-L2OB.reshape(50,4)))#L2OB#-myOB
print('#'*30+'L2 DATA LOB for Horizon: '+ str(horizon)+' '+'#'*30)
print(L2OB.reshape(50,4))
print('#'*30+'JAX LOB for Horizon: '+ str(horizon)+' '+'#'*30)
print(myOB.reshape(150,4)[:,:])


[        1        -1       100   2167900         0  35437285     36205
 906644796]
##############################JAX - L2 DATA LOB Difference for Horizon: 14136 ##############################
[[      0       0       0       0]
 [      0       0       0       0]
 [      0       0    -100     200]
 [      0       0   -1500    -307]
 [      0       0   -1000    -950]
 [      0       0    -900     100]
 [      0       0   -2100      50]
 [      0       0   -2900       0]
 [      0       0   -2700    -100]
 [      0       0   -6800      90]
 [      0       0   -6900     300]
 [      0       0  -10600     -81]
 [      0       0  -10100     -20]
 [      0       0  -10800    -446]
 [      0       0  -13900       0]
 [      0       0  -25400     190]
 [      0       0  -26600      60]
 [      0       0  -30400       0]
 [      0       0  -35300      90]
 [      0       0  -54200      49]
 [      0       0  -55000    -250]
 [      0       0  -56200     -55]
 [      0       0  -63600      99]
 [ 

In [305]:
#Price: 2160000  Order ID: 33737046  Quant: 30 Type: DELETE

In [313]:
print(orderbook_2.bids.get_price_list(2160000))
orderbook_1.orderbook_array[1,7,:,:]


Quantity   100  |  Price  2160000  |  Order_ID   35268348  |  Time 36194.069208923



Array([[     100,  2160000,        0, 35268348,    36194, 69208923],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       

## Load the original orderbook implementation on the CPU

In [315]:
orderbook_2=cpuOrderBook()

In [316]:
limit_orders = []
order_id_list = [orderbook.orderbook.INITID + i for i in range(100)]
for i in range(50):
    trade_id = 10086
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'ask', 
          'quantity' : df_ob.iloc[0,4* i + 1], 
          'price' : df_ob.iloc[0,4*i],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
for i in range(50):
    trade_id = 10087
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'bid', 
          'quantity' : df_ob.iloc[0,4* i + 3], 
          'price' : df_ob.iloc[0,4*i+2],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
# Add init orders to order book
for order in limit_orders:
    # breakpoint()
    trades, order_id = orderbook_2.process_order(order, True, False)  

In [317]:
##Taken largely from the Example.py file in the orderbook repository for the cpu

t1=time.time()
for index in range(horizon):
    l1 = df_msg.iloc[index,:]
    ttype = l1[1] 
    side = 'bid' if l1[5] ==1 else 'ask'
    quantity = l1[3]
    price = l1[4]
    trade_id = l1[2] # not sure, in the data it is order id
    order_id = trade_id
    timestamp = str(l1[0])
    message = {'type': ttype,'side': side,'quantity': quantity,'price': price,'trade_id': trade_id, "timestamp":timestamp, 'order_id':order_id}
    orderbook_2.processOrder(message,True,False)

t2=time.time()
        
print(message)

got to the ask arm
got to the ask arm
got to the no ID issue
Quantity    40  |  Price  2257500  |  Order_ID   15000009  |  Time 34200.000000001

Trying to cancel partial order
{'type': 'delete', 'side': 'ask', 'quantity': 20, 'price': 2257500, 'trade_id': 5097802, 'timestamp': '34200.501657651', 'order_id': 15000009}
got to the ask arm
got to the no ID issue
Quantity   300  |  Price  2237200  |  Order_ID   15000002  |  Time 34200.000000001

Deleting whole order
got to the bid arm
got to the no ID issue
Quantity   300  |  Price  2226400  |  Order_ID   15000000  |  Time 34200.000000001

Deleting whole order
got to the bid arm
got to the no ID issue
Quantity   200  |  Price  2222200  |  Order_ID   15000001  |  Time 34200.000000001
Quantity     1  |  Price  2222200  |  Order_ID    2584610  |  Time 34200.078873974

Deleting whole order
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got t

got to the ask arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the b

got to the bid arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the b

got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the no ID issue
Couldn't find the price in the price list
got to the ask arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the bid arm
got to the ask arm
got to the bid arm
got to the ask arm
got 

In [318]:
t2-t1

0.8085677623748779

In [319]:
from itertools import zip_longest
cpuOB=jnp.array(list(zip_longest(*orderbook_2.get_L2_state(), fillvalue=-1)))

In [320]:
print('#'*30+'CPU - JAX LOB Difference for Horizon: '+ str(horizon)+' '+'#'*30)
print((cpuOB[:50,:]-myOB.reshape(150,4)[0:50,:].astype('int32')))
print('#'*30+'CPU LOB for Horizon: '+ str(horizon)+' '+'#'*30)
print(cpuOB[:50,:])
print('#'*30+'JAX LOB for Horizon: '+ str(horizon)+' '+'#'*30)
print(myOB.reshape(150,4)[0:100,:])


##############################CPU - JAX LOB Difference for Horizon: 14137 ##############################
[[  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0 -70]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [  0   0   0   0]
 [

In [373]:
limit_orders = []
order_id_list = [15000000 + i for i in range(10)]
trade_id = 10086
# timestamp = datetime(34200.000000001)
timestamp = str(34200.000000001)
item = {'type' : 'limit', 
    'side' : 'bid', 
      'quantity' : 100, 
      'price' : 2226400,
      'trade_id' : trade_id,
      'order_id' : order_id_list[0],
      "timestamp": timestamp}

SyntaxError: invalid syntax (<unknown>, line 1)

In [396]:
orderbook_1.process_order(item)

(Array([[      -1,       -1,       -1, 15000000,       -1,    34200],
        [      -1,       -1,       -1, 15000000,       -1,    34200],
        [      -1,       -1,       -1, 15000000,       -1,    34200],
        [      -1,       -1,       -1, 15000000,       -1,    34200],
        [      -1,       -1,       -1, 15000000,       -1,    34200]],      dtype=int32),
 Array([       1,        1,      100,  2226400,    10086, 15000000,
           34200,        1], dtype=int32))

In [397]:
orderbook_1.get_L2_state()

Array([2229100,    1507, 2226400,    1500, 2234000,      50, 2222200,
           200, 2237200,     300, 2221500,     100, 2237500,     100,
       2221400,      20, 2240000,     200, 2221000,      14, 2242500,
            20, 2218300,      20, 2247500,      20, 2210000,     100,
       2250000,     100, 2203200,     100, 2252500,      20, 2200000,
            97, 2257500,      40, 2190000,      50, 2262700,       7,
       2185100,       1, 2264900,       1, 2180000,       4, 2270000,
             5, 2153200,     100, 2290000,      70, 2150700,       1,
       2299000,     100, 2133600,     300, 2300000,     139, 2132600,
           100, 2301400,     100, 2100900,      50, 2318300,     100,
       2100000,      50, 2318500,     300, 2096000,      15, 2330000,
            50, 2088200,     100, 2340000,      50, 2078000,     300,
       2344300,       5, 2066900,     100, 2362900,     100, 2050000,
            75, 2364200,      60, 2030000,      10, 2374300,     300,
       2000000,     

In [362]:
item

{'type': 'limit',
 'side': 'bid',
 'quantity': 100,
 'price': 280000,
 'trade_id': 10086,
 'order_id': 15000000,
 'timestamp': '34200.000000001'}